In [224]:
import time
import json
import os
from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen
from collections import defaultdict
import re
import pandas as pd
import numpy as np

CORPUS = pd.read_csv("data/final_corpus.csv", encoding="utf-8").set_index("id")
METADATA = pd.read_csv(
    "data/lang-8-users.csv",
    encoding="utf-8",
    names=[
        "sex",
        "occupation",
        "lpoints",
        "user_id",
        "nation_region",
        "location",
        "age",
    ],
    dtype={"user_id": str}
)

In [214]:
filtered_lst = ["cho", "hamtaro", "mayukichi", "arekusei", "achamin", "seroris", "fususu", "siut_aro", "stardust"]

In [215]:
CORPUS = CORPUS.query("user_id not in @filtered_lst")

In [30]:
sample = {
   "corpusType": 1,  #0-unannotated, 1-annotated, -1 -both (any)
   "agreementType": 1, #0 = 3 no's, 1 = 1 yes, 2 no's, 2 = 2 yeses 1 no, 3 = 3 yeses (if agreementType=-1, means any agreement type)
   "sentenceType": 0, #0-original, 1-corrected, -1 -both (any)
   "wordRange": [10, 20], #based on 'sentenceType', sentences with word range in 10-20 inclusively (if wordRange = -1, means any word range)
   "nCorrections": 2, #sentences with n corrections (-1 means any)
   "sex": 0, #0-male, 1-female, 2-other (-1 means any)
   "occupation": "tudent", #should we have a list of fixed occupations or string is ok? ("all" means all :) )
   "location": "Canada", #should we have a list of fixed locations or string is ok? ("all" means all :) )
   "Lpoints": [100, 200] #Lpoints in the given range inclusively ("all" means any)
}

In [31]:
def get_dict_corpus(size = 10):
    return CORPUS[:size].T.to_dict('list')

def get_by_range(sent_type, range = (10, 15)):
    mask =  (CORPUS[sent_type].str.len() >= range[0]) & (CORPUS[sent_type].str.len() <= range[1])
    return CORPUS.loc[mask].T.to_dict('list')

In [53]:
import pdb
def get_by_metadata(args):
    print(args['sex'])
    pdb.set_trace()
    combined = pd.merge(CORPUS, METADATA, on="user_id")
    
    query_str = " & ".join([f"{k} = {v}" for k, v in args.items()])
    
    combined.query(query_str)

In [51]:
def construct_metadata_dict(param_dict):
    metadata_key = {"sex", "occupation", "location", "Lpoints"}
    new_dict = {}
    for k, v in param_dict.items():
        if k in metadata_key:
            if k == "sex":
                if v == 0:
                    v = "Male"
                elif v == 1:
                    v = "Female"
                else:
                    continue
            else:
                if v == "all":
                    continue
            
            new_dict[k] = v 
    return new_dict

In [55]:
metadata_d = construct_metadata_dict(sample)
get_by_metadata(metadata_d)

Male
> <ipython-input-53-6ee80d559396>(5)get_by_metadata()
      3     print(args['sex'])
      4     pdb.set_trace()
----> 5     combined = pd.merge(CORPUS, METADATA, on="user_id")
      6 
      7     query_str = " & ".join([f"{k} = {v}" for k, v in args.items()])



ipdb>  n


ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat
> <ipython-input-53-6ee80d559396>(5)get_by_metadata()
      3     print(args['sex'])
      4     pdb.set_trace()
----> 5     combined = pd.merge(CORPUS, METADATA, on="user_id")
      6 
      7     query_str = " & ".join([f"{k} = {v}" for k, v in args.items()])



ipdb>  q


BdbQuit: 

In [20]:
popular_occupation = [
    "Artist",
    "Designer",
    "Engineer",
    "Housewife/ Househusband",
    "Office worker",
    "Programmer",
    "Student",
    "Teacher",
    "Other"
]